# Multi-Model Bongard Problem Difficulty Analysis

Analyzes all model results to find:
- **Top 100 worst performing problems** (lowest success rate across models)
- **Models that did well in top 100 worst performing problems**
- Overall model comparison

In [32]:
import os
import json
import pandas as pd
from collections import defaultdict
import glob

In [33]:
# Configuration - look in results directory
results_dir = "../results"
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def analyze_all_models(results_dir):
    """Analyze all JSON files in results directory."""
    
    # Find all JSON files in results directory
    json_files = glob.glob(os.path.join(results_dir, "*.json"))
    print(f"Found {len(json_files)} JSON files in {results_dir}")
    
    if not json_files:
        print("No JSON files found!")
        return {}, {}
    
    # Track results per problem and per model
    problem_results = defaultdict(lambda: {'correct': 0, 'total': 0, 'models_correct': [], 'models_wrong': []})
    model_accuracy = {}
    
    for file_path in json_files:
        model_name = os.path.basename(file_path).replace('.json', '')
        
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
            
            # Check if data is a list
            if not isinstance(data, list):
                continue
                
            correct = 0
            processed = 0
            
            for entry in data:
                # Check if entry is a dict and has required fields
                if not isinstance(entry, dict):
                    continue
                if 'uid' not in entry or 'answer' not in entry:
                    continue
                    
                uid = entry['uid']
                answer = entry['answer']
                
                # Determine expected answer based on UID suffix
                if uid.endswith('A'):
                    expected = 'positive'
                elif uid.endswith('B'):
                    expected = 'negative'
                else:
                    continue
                
                processed += 1
                
                # Track per problem
                problem_results[uid]['total'] += 1
                if answer == expected:
                    correct += 1
                    problem_results[uid]['correct'] += 1
                    problem_results[uid]['models_correct'].append(model_name)
                else:
                    problem_results[uid]['models_wrong'].append(model_name)
            
            if processed > 0:
                # Track per model
                accuracy = (correct / processed) * 100
                model_accuracy[model_name] = {
                    'correct': correct,
                    'total': processed,
                    'accuracy': accuracy
                }
                
        except Exception as e:
            pass  # Silently skip errors
    
    return problem_results, model_accuracy

def find_worst_problems(problem_results, top_n=100):
    """Find the worst performing problems."""
    
    if not problem_results:
        return pd.DataFrame()
    
    problem_stats = []
    for uid, stats in problem_results.items():
        success_rate = (stats['correct'] / stats['total']) * 100 if stats['total'] > 0 else 0
        problem_stats.append({
            'UID': uid,
            'Success_Rate': success_rate,
            'Correct': stats['correct'],
            'Total': stats['total'],
            'Models_Correct': ', '.join(stats['models_correct']) if stats['models_correct'] else 'None'
        })
    
    # Sort by success rate (worst first)
    problem_stats.sort(key=lambda x: x['Success_Rate'])
    
    return pd.DataFrame(problem_stats[:top_n])


In [34]:
# Run the analysis
problem_results, model_accuracy = analyze_all_models(results_dir)

if model_accuracy:
    print(f"MODEL ACCURACY SUMMARY:")
    print("-" * 50)
    model_df = pd.DataFrame.from_dict(model_accuracy, orient='index').sort_values('accuracy')
    display(model_df)

    # Find worst performing problems
    worst_problems = find_worst_problems(problem_results, top_n=100)

    if not worst_problems.empty:
        # Add SCMR column
        worst_problems['SDR_GPT41_Correct'] = worst_problems['Models_Correct'].apply(
            lambda x: 'Yes' if 'sdr_gpt41_gpt41' in str(x) else 'No'
        )
        
        print(f"\nTOP 100 WORST PERFORMING PROBLEMS:")
        print("-" * 80)
        
        # Show all rows without truncation
        # with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
        #     display(worst_problems)
        
    else:
        print("No problems to analyze!")
else:
    print("No valid model results found!")


Found 38 JSON files in ../results
MODEL ACCURACY SUMMARY:
--------------------------------------------------


,correct,total,accuracy
ivr_gpt5,0,1,0.00
sdr_gpt41_llama4,215,400,53.75
cmr_llava16_gemma3,241,400,60.25
sdr_gpt41_qwen3,244,400,61.00
cmr_llama4_gemma3,250,400,62.50
cmr_llava16_llama4,253,400,63.25
cmr_llava16_qwen3,253,400,63.25
cvr_mistral3,254,400,63.50
cmr_llava16_deepseekr1,255,400,63.75
sdr_gpt41_deepseekr1,256,400,64.00



TOP 100 WORST PERFORMING PROBLEMS:
--------------------------------------------------------------------------------


In [35]:
# %%
# Which models perform best on the worst 100 problems
if not worst_problems.empty and problem_results:
    worst_100_uids = set(worst_problems['UID'].tolist())
    
    # Track each model's performance on worst problems
    model_performance_on_worst = defaultdict(lambda: {'correct': 0, 'total': 0})
    
    for uid in worst_100_uids:
        if uid in problem_results:
            stats = problem_results[uid]
            
            for model in stats['models_correct']:
                model_performance_on_worst[model]['correct'] += 1
                model_performance_on_worst[model]['total'] += 1
            
            for model in stats['models_wrong']:
                model_performance_on_worst[model]['total'] += 1
    
    # Create results
    results = []
    for model, stats in model_performance_on_worst.items():
        if stats['total'] > 0:
            success_rate = (stats['correct'] / stats['total']) * 100
            results.append({
                'Model': model,
                'Correct': stats['correct'],
                'Total': stats['total'],
                'Success_Rate': round(success_rate, 2)
            })
    
    # Sort by success rate and display
    results.sort(key=lambda x: x['Success_Rate'], reverse=True)
    performance_df = pd.DataFrame(results)
    
    print("MODEL PERFORMANCE ON WORST 100 PROBLEMS:")
    display(performance_df)
else:
    print("No data available for analysis")


MODEL PERFORMANCE ON WORST 100 PROBLEMS:


,Model,Correct,Total,Success_Rate
0,sdr_gpt41_gptoss,37,100,37.0
1,sdr_gpt41_deepseekr1,36,100,36.0
2,cvr_gpt5,36,100,36.0
3,cmr_llama4_qwen3,34,100,34.0
4,sdr_gpt41_qwen3,33,100,33.0
5,sdr_gpt41_llama4,32,100,32.0
6,cvr_gemini25,31,100,31.0
7,cmr_gpt5_gpt5,31,100,31.0
8,sdr_gpt41_gpt41,30,100,30.0
9,cmr_gpt41_gptoss,29,100,29.0


In [36]:
# %%
# Find problems that sdr_gpt41_gpt41 got wrong

if problem_results:
    sdr_wrong_problems = []
    ll = []
    for uid, stats in problem_results.items():
        if 'sdr_gpt41_gpt41' in stats['models_wrong']:
            cvr_correct = 'Yes' if 'cvr_gpt41' in stats['models_correct'] else 'No'
            scmr2_correct = 'Yes' if 'scmr2_gpt41_gpt41' in stats['models_correct'] else 'No'
            if cvr_correct == 'Yes' and scmr2_correct == 'Yes':
                ll.append(uid)
            sdr_wrong_problems.append({
                'UID': uid,
                'CVR_GPT41_Correct': cvr_correct,
                'SCMR2_GPT41_Correct': scmr2_correct,

            })
    print(ll.sort())
    print(len(ll))
    
    if sdr_wrong_problems:
        sdr_wrong_df = pd.DataFrame(sdr_wrong_problems).sort_values('UID').reset_index(drop=True)
        
        print(f"PROBLEMS THAT SDR_GPT41_GPT41 GOT WRONG ({len(sdr_wrong_problems)} total):")
        print("-" * 60)
        
        # Define styling function
        def highlight_both_correct(row):
            if row['CVR_GPT41_Correct'] == 'Yes' and row['SCMR2_GPT41_Correct'] == 'Yes':
                return ['background-color: purple'] * len(row)
            else:
                return [''] * len(row)
        
        with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
            display(sdr_wrong_df.style.apply(highlight_both_correct, axis=1))
        
    else:
        print("SDR_GPT41_GPT41 got all problems correct!")
else:
    print("No valid problem results found!")

None
0
PROBLEMS THAT SDR_GPT41_GPT41 GOT WRONG (107 total):
------------------------------------------------------------


,UID,CVR_GPT41_Correct,SCMR2_GPT41_Correct
0,0008_B,No,No
1,0014_B,Yes,No
2,0025_A,No,No
3,0025_B,No,No
4,0037_B,No,No
5,0044_A,No,No
6,0044_B,Yes,No
7,0057_B,No,No
8,0062_A,Yes,No
9,0080_A,Yes,No


In [37]:
# Find all problems that GPT-5 got wrong
if problem_results:
    gpt5_wrong_problems = []
    
    for uid, stats in problem_results.items():
        if 'cvr_gpt5' in stats['models_wrong']:
            # Check other models' performance on this problem
            cvr_gpt41_correct = 'Yes' if 'cvr_gpt41' in stats['models_correct'] else 'No'
            sdr_gpt41_correct = 'Yes' if 'sdr_gpt41_gpt41' in stats['models_correct'] else 'No'
            cmr_gpt5_correct = 'Yes' if 'cmr_gpt5_gpt5' in stats['models_correct'] else 'No'
            
            gpt5_wrong_problems.append({
                'UID': uid,
                'CVR_GPT41_Correct': cvr_gpt41_correct,
                'SDR_GPT41_Correct': sdr_gpt41_correct,
                'CMR_GPT5_Correct': cmr_gpt5_correct,
                'Success_Rate': round((stats['correct'] / stats['total']) * 100, 1) if stats['total'] > 0 else 0
            })
    
    if gpt5_wrong_problems:
        gpt5_wrong_df = pd.DataFrame(gpt5_wrong_problems).sort_values('Success_Rate').reset_index(drop=True)
        
        print(f"PROBLEMS THAT GPT-5 (CVR) GOT WRONG ({len(gpt5_wrong_problems)} total):")
        print("-" * 70)
        
        # Show summary statistics
        cvr_gpt41_better = sum(1 for p in gpt5_wrong_problems if p['CVR_GPT41_Correct'] == 'Yes')
        sdr_gpt41_better = sum(1 for p in gpt5_wrong_problems if p['SDR_GPT41_Correct'] == 'Yes')
        cmr_gpt5_better = sum(1 for p in gpt5_wrong_problems if p['CMR_GPT5_Correct'] == 'Yes')
        
        print(f"Of the {len(gpt5_wrong_problems)} problems GPT-5 (CVR) got wrong:")
        print(f"- CVR GPT-4.1 got {cvr_gpt41_better} correct ({cvr_gpt41_better/len(gpt5_wrong_problems)*100:.1f}%)")
        print(f"- SDR GPT-4.1 got {sdr_gpt41_better} correct ({sdr_gpt41_better/len(gpt5_wrong_problems)*100:.1f}%)")
        print(f"- CMR GPT-5 got {cmr_gpt5_better} correct ({cmr_gpt5_better/len(gpt5_wrong_problems)*100:.1f}%)")
        print()
        
        with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
            display(gpt5_wrong_df)
        
    else:
        print("GPT-5 (CVR) got all problems correct!")
else:
    print("No valid problem results found!")

PROBLEMS THAT GPT-5 (CVR) GOT WRONG (81 total):
----------------------------------------------------------------------
Of the 81 problems GPT-5 (CVR) got wrong:
- CVR GPT-4.1 got 14 correct (17.3%)
- SDR GPT-4.1 got 28 correct (34.6%)
- CMR GPT-5 got 18 correct (22.2%)



,UID,CVR_GPT41_Correct,SDR_GPT41_Correct,CMR_GPT5_Correct,Success_Rate
0,0159_B,No,No,No,0.0
1,0789_B,No,No,No,0.0
2,0094_B,No,No,No,0.0
3,0640_B,No,No,No,0.0
4,0739_A,No,No,No,0.0
5,0713_B,No,No,No,0.0
6,0829_A,No,No,No,2.7
7,0386_B,No,No,No,2.7
8,0025_B,No,No,No,2.7
9,0753_B,No,No,No,2.7
